In [1]:
# Fuse Videos
import cv2
import os
import numpy as np

In [2]:
# Get the video folder of a trial
video_path_trial = r'/home/yiting/Documents/Data/Videos/2024-04-15/camera_annotation/2024-04-15_10-36-24_969748'
cameras = ["camTL", "camTo", "camTR", "camBL", "camBo", "camBR"]

# Get the frame size and total frame number
vidcap = cv2.VideoCapture(os.path.join(video_path_trial, cameras[0] + '.mp4'))
success, frame = vidcap.read()
height, width, layers = frame.shape
total_frame_number = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(vidcap.get(cv2.CAP_PROP_FPS))


In [3]:
downsample = 1
resize_factor = 10
new_width = int(width/resize_factor)
new_height = int(height/resize_factor)
# Initiate a video for combined videos
video_save_path = r'/home/yiting/Documents/Data/Videos/FusedVideos/2024-04-15_10-36-24_969748_resized10_downsampled2.mp4'
video = cv2.VideoWriter(video_save_path,  
                fourcc = cv2.VideoWriter_fourcc(*'mp4v'), 
                fps=frame_rate/downsample,                                       
                frameSize=(new_width*3, new_height*2)) 

for idx in range(total_frame_number):
    if idx % downsample == 0:
        frames = dict()
        for c in cameras:
            vidcap = cv2.VideoCapture(os.path.join(video_path_trial, c + '.mp4'))
            vidcap.set(cv2.CAP_PROP_POS_FRAMES, idx) # Set the frame to get
            success, frame = vidcap.read() 
            frames[c] = cv2.resize(frame, (new_height, new_width))

        # Generage top-row frame by concatenating camTL, camTo, camTR horizontally
        combined_frames_top = np.concatenate((frames["camTL"], frames["camTo"], frames["camTR"]), axis=1)

        # Generate bottom-row image by concatenating camBL, camBo, camBR horizontally
        combined_frames_bottom = np.concatenate((frames["camBL"], frames["camBo"], frames["camBR"]), axis=1)

        # Concatenate top-row and bottom-row images vertically
        combined_frames = np.concatenate((combined_frames_top, combined_frames_bottom), axis=0)

        # Write the combined image to the video
        video.write(combined_frames)
    
cv2.destroyAllWindows()    
video.release()